In [1]:
import pandas as pd
import pandas_profiling as pp


In [2]:
import os
data_path = '/home/srgrace/genericContest_data/cat-in-the-dat-ii'

train = pd.read_csv(os.path.join(data_path, 'train.csv'))
test = pd.read_csv(os.path.join(data_path, 'test.csv'))

In [3]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [4]:
test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,...,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0
1,600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,...,060a21580,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0
2,600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,...,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0
3,600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,...,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0
4,600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,...,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0


In [5]:
import warnings
warnings.filterwarnings("ignore")

import gc
import joblib
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [6]:
test['target'] = -1
data = pd.concat([train, test]).reset_index(drop=True)

features = [x for x in train.columns if x not in ['id', 'target']]

for feat in features:
    label_enc = preprocessing.LabelEncoder()
    data[feat] = label_enc.fit_transform(data[feat].fillna('-1').astype(str).values)

In [7]:
train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)

In [8]:
train.tail()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
599995,599995,1,2,1,2,1,3,2,1,5,...,14,3,5,3,1,18,23,5,0,0
599996,599996,2,1,1,2,2,1,2,3,3,...,1403,2,5,1,14,14,164,0,6,0
599997,599997,1,1,1,1,2,3,1,1,6,...,572,2,1,3,14,8,43,7,8,0
599998,599998,2,2,1,1,2,0,2,1,0,...,1712,1,4,6,13,24,19,1,8,0
599999,599999,1,1,1,2,1,1,6,3,6,...,1974,1,1,1,2,15,168,5,11,0


In [9]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [16]:
def create_model(data, feats):
    inps = []
    outs = []
    for feat in feats:
        num_unique_vals = int(data[feat].nunique())
        embed_dim = int(min(np.ceil(num_unique_vals/2), 50))
        
        inp = layers.Input(shape=(1, ))
        out = layers.Embedding(num_unique_vals + 1, embed_dim, name=feat)(inp)
        
        out = layers.SpatialDropout1D(0.5)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        
        inps.append(inp)
        outs.append(out)
    
    x = layers.Concatenate()(outs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation='softmax')(x)
    
    model = Model(inputs=inps, outputs=y)
    return model

In [17]:
test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]
oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))


skf = StratifiedKFold(n_splits=50)
for train_idx, test_idx in skf.split(train, train.target.values):
    
    x_train, x_test = train.iloc[train_idx, :], train.iloc[test_idx, :]
    x_train = x_train.reset_index(drop=True)
    x_test = x_test.reset_index(drop=True)
    
    y_train, y_test = x_train.target.values, x_test.target.values
    
    model = create_model(data, features)
    model.compile(loss='binary_crossentropy', optimizers='adam', metrics=[auc])
    
    x_train = [x_train.loc[:, features].values[:, k] for k in range(x_train.loc[:, features].values.shape[1])]
    x_test = [x_test.loc[:, features].values[:, k] for k in range(x_test.loc[:, features].values.shape[1])]
    
    early_stop = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=5, verbose=1, mode='max',
                                         baseline=None, restore_best_weights=True)
    
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=3, min_lr=1e-6, 
                                           mode='max', verbose=1)
    
    model.fit(x_train, utils.to_categorical(y_train),
             validation_data=(x_test, utils.to_categorical(y_test)),
             verbose=1, batch_size=1024, callbacks=[early_stop, reduce_lr], epochs=100)
    
    valid_fold_preds = model.predict(x_test)[:, 1]
    test_fold_preds = model.predict(test_data)[:, 1]
    oof_preds[test_idx] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()
    print(metrics.roc_auc_score(y_test, valid_fold_preds))
    K.clear_session()

Train on 587999 samples, validate on 12001 samples
Epoch 1/100
587999/587999 [==============================] - 43s 74us/sample - loss: 0.4674 - auc: 0.6911 - val_loss: 0.4075 - val_auc: 0.7756
Epoch 2/100
587999/587999 [==============================] - 22s 37us/sample - loss: 0.4159 - auc: 0.7575 - val_loss: 0.4073 - val_auc: 0.7768
Epoch 3/100
587999/587999 [==============================] - 19s 33us/sample - loss: 0.4121 - auc: 0.7641 - val_loss: 0.4027 - val_auc: 0.7798
Epoch 4/100
587999/587999 [==============================] - 19s 32us/sample - loss: 0.4107 - auc: 0.7666 - val_loss: 0.4017 - val_auc: 0.7797
Epoch 5/100
587999/587999 [==============================] - 19s 32us/sample - loss: 0.4096 - auc: 0.7682 - val_loss: 0.4022 - val_auc: 0.7801
Epoch 6/100
587999/587999 [==============================] - 20s 35us/sample - loss: 0.4088 - auc: 0.7695 - val_loss: 0.4018 - val_auc: 0.7801
Epoch 7/100
587999/587999 [==============================] - 22s 38us/sample - loss: 0.4085

Epoch 00008: early stopping
0.7809423340659987
Train on 587999 samples, validate on 12001 samples
Epoch 1/100
587999/587999 [==============================] - 23s 40us/sample - loss: 0.4670 - auc: 0.6915 - val_loss: 0.4071 - val_auc: 0.7818
Epoch 2/100
587999/587999 [==============================] - 20s 35us/sample - loss: 0.4164 - auc: 0.7566 - val_loss: 0.3986 - val_auc: 0.7848
Epoch 3/100
587999/587999 [==============================] - 24s 40us/sample - loss: 0.4126 - auc: 0.7632 - val_loss: 0.4033 - val_auc: 0.7846
Epoch 4/100
587999/587999 [==============================] - 23s 39us/sample - loss: 0.4108 - auc: 0.7661 - val_loss: 0.4009 - val_auc: 0.7857
Epoch 5/100
587999/587999 [==============================] - 23s 39us/sample - loss: 0.4094 - auc: 0.7684 - val_loss: 0.3977 - val_auc: 0.7855
Epoch 6/100
587999/587999 [==============================] - 27s 45us/sample - loss: 0.4089 - auc: 0.7692 - val_loss: 0.4021 - val_auc: 0.7859
Epoch 7/100
587999/587999 [=================

Train on 587999 samples, validate on 12001 samples
Epoch 1/100
587999/587999 [==============================] - 32s 55us/sample - loss: 0.4726 - auc: 0.6858 - val_loss: 0.4035 - val_auc: 0.7803
Epoch 2/100
587999/587999 [==============================] - 25s 43us/sample - loss: 0.4158 - auc: 0.7578 - val_loss: 0.4003 - val_auc: 0.7838
Epoch 3/100
587999/587999 [==============================] - 25s 42us/sample - loss: 0.4123 - auc: 0.7637 - val_loss: 0.3990 - val_auc: 0.7842
Epoch 4/100
587999/587999 [==============================] - 25s 42us/sample - loss: 0.4111 - auc: 0.7661 - val_loss: 0.4000 - val_auc: 0.7852
Epoch 5/100
587999/587999 [==============================] - 25s 42us/sample - loss: 0.4100 - auc: 0.7676 - val_loss: 0.3981 - val_auc: 0.7849
Epoch 6/100
587999/587999 [==============================] - 25s 42us/sample - loss: 0.4091 - auc: 0.7689 - val_loss: 0.4069 - val_auc: 0.7852
Epoch 7/100
587776/587999 [============================>.] - ETA: 0s - loss: 0.4084 - auc: 

Epoch 00010: early stopping
0.7760094816691775
Train on 587999 samples, validate on 12001 samples
Epoch 1/100
587999/587999 [==============================] - 32s 55us/sample - loss: 0.4683 - auc: 0.6920 - val_loss: 0.4067 - val_auc: 0.7601
Epoch 2/100
587999/587999 [==============================] - 24s 41us/sample - loss: 0.4160 - auc: 0.7574 - val_loss: 0.4001 - val_auc: 0.7622
Epoch 3/100
587999/587999 [==============================] - 24s 41us/sample - loss: 0.4128 - auc: 0.7631 - val_loss: 0.4047 - val_auc: 0.7630
Epoch 4/100
587999/587999 [==============================] - 25s 42us/sample - loss: 0.4109 - auc: 0.7660 - val_loss: 0.3969 - val_auc: 0.7641
Epoch 5/100
587999/587999 [==============================] - 24s 42us/sample - loss: 0.4099 - auc: 0.7677 - val_loss: 0.3982 - val_auc: 0.7638
Epoch 6/100
587999/587999 [==============================] - 24s 42us/sample - loss: 0.4088 - auc: 0.7693 - val_loss: 0.3973 - val_auc: 0.7645
Epoch 7/100
587999/587999 [=================

587999/587999 [==============================] - 23s 39us/sample - loss: 0.4124 - auc: 0.7639 - val_loss: 0.4050 - val_auc: 0.7533
Epoch 4/100
587999/587999 [==============================] - 23s 39us/sample - loss: 0.4107 - auc: 0.7668 - val_loss: 0.4046 - val_auc: 0.7545
Epoch 5/100
587999/587999 [==============================] - 23s 39us/sample - loss: 0.4099 - auc: 0.7678 - val_loss: 0.4033 - val_auc: 0.7544
Epoch 6/100
587999/587999 [==============================] - 23s 39us/sample - loss: 0.4089 - auc: 0.7693 - val_loss: 0.4053 - val_auc: 0.7542
Epoch 7/100
586752/587999 [============================>.] - ETA: 0s - loss: 0.4082 - auc: 0.7704
Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
587999/587999 [==============================] - 23s 39us/sample - loss: 0.4082 - auc: 0.7703 - val_loss: 0.4041 - val_auc: 0.7544
Epoch 8/100
587999/587999 [==============================] - 23s 39us/sample - loss: 0.4067 - auc: 0.7722 - val_loss: 0.4060 - val_

Epoch 4/100
588000/588000 [==============================] - 25s 42us/sample - loss: 0.4110 - auc: 0.7660 - val_loss: 0.4002 - val_auc: 0.7384
Epoch 5/100
588000/588000 [==============================] - 24s 42us/sample - loss: 0.4096 - auc: 0.7684 - val_loss: 0.3978 - val_auc: 0.7388
Epoch 6/100
588000/588000 [==============================] - 24s 41us/sample - loss: 0.4088 - auc: 0.7698 - val_loss: 0.3997 - val_auc: 0.7375
Epoch 7/100
588000/588000 [==============================] - 24s 41us/sample - loss: 0.4088 - auc: 0.7694 - val_loss: 0.4122 - val_auc: 0.7373
Epoch 00007: early stopping
0.7869795545668321
Train on 588000 samples, validate on 12000 samples
Epoch 1/100
588000/588000 [==============================] - 32s 55us/sample - loss: 0.4690 - auc: 0.6903 - val_loss: 0.4064 - val_auc: 0.7593
Epoch 2/100
588000/588000 [==============================] - 25s 42us/sample - loss: 0.4159 - auc: 0.7572 - val_loss: 0.4021 - val_auc: 0.7555
Epoch 3/100
588000/588000 [=================

588001/588001 [==============================] - 24s 41us/sample - loss: 0.4121 - auc: 0.7642 - val_loss: 0.3985 - val_auc: 0.7611
Epoch 4/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4106 - auc: 0.7668 - val_loss: 0.3963 - val_auc: 0.7627
Epoch 5/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4096 - auc: 0.7680 - val_loss: 0.3975 - val_auc: 0.7622
Epoch 6/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4092 - auc: 0.7686 - val_loss: 0.3999 - val_auc: 0.7619
Epoch 7/100
587776/588001 [============================>.] - ETA: 0s - loss: 0.4082 - auc: 0.7704
Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4082 - auc: 0.7705 - val_loss: 0.3966 - val_auc: 0.7625
Epoch 8/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4066 - auc: 0.7723 - val_loss: 0.3964 - val_

588001/588001 [==============================] - 32s 55us/sample - loss: 0.4688 - auc: 0.6903 - val_loss: 0.4102 - val_auc: 0.7821
Epoch 2/100
588001/588001 [==============================] - 24s 40us/sample - loss: 0.4157 - auc: 0.7581 - val_loss: 0.4028 - val_auc: 0.7877
Epoch 3/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4121 - auc: 0.7641 - val_loss: 0.4029 - val_auc: 0.7846
Epoch 4/100
588001/588001 [==============================] - 25s 42us/sample - loss: 0.4109 - auc: 0.7663 - val_loss: 0.4022 - val_auc: 0.7856
Epoch 5/100
587776/588001 [============================>.] - ETA: 0s - loss: 0.4094 - auc: 0.7685
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
588001/588001 [==============================] - 25s 42us/sample - loss: 0.4094 - auc: 0.7685 - val_loss: 0.3997 - val_auc: 0.7875
Epoch 6/100
588001/588001 [==============================] - 25s 42us/sample - loss: 0.4078 - auc: 0.7706 - val_loss: 0.4005 - val_

588001/588001 [==============================] - 25s 42us/sample - loss: 0.4068 - auc: 0.7719 - val_loss: 0.3984 - val_auc: 0.7598
Epoch 00007: early stopping
0.7861108607904647
Train on 588001 samples, validate on 11999 samples
Epoch 1/100
588001/588001 [==============================] - 31s 54us/sample - loss: 0.4682 - auc: 0.6916 - val_loss: 0.4137 - val_auc: 0.7553
Epoch 2/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4159 - auc: 0.7574 - val_loss: 0.4043 - val_auc: 0.7593
Epoch 3/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4120 - auc: 0.7641 - val_loss: 0.4032 - val_auc: 0.7588
Epoch 4/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4106 - auc: 0.7667 - val_loss: 0.4015 - val_auc: 0.7600
Epoch 5/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4097 - auc: 0.7682 - val_loss: 0.4006 - val_auc: 0.7588
Epoch 6/100
588001/588001 [=============================

Epoch 4/100
588001/588001 [==============================] - 24s 40us/sample - loss: 0.4110 - auc: 0.7660 - val_loss: 0.3987 - val_auc: 0.7894
Epoch 5/100
588001/588001 [==============================] - 24s 41us/sample - loss: 0.4094 - auc: 0.7683 - val_loss: 0.3961 - val_auc: 0.7914
Epoch 6/100
588001/588001 [==============================] - 25s 42us/sample - loss: 0.4093 - auc: 0.7684 - val_loss: 0.3964 - val_auc: 0.7904
Epoch 7/100
588001/588001 [==============================] - 25s 42us/sample - loss: 0.4083 - auc: 0.7701 - val_loss: 0.3972 - val_auc: 0.7898
Epoch 00007: early stopping
0.7892555424414929
Train on 588001 samples, validate on 11999 samples
Epoch 1/100
588001/588001 [==============================] - 32s 55us/sample - loss: 0.4677 - auc: 0.6891 - val_loss: 0.4096 - val_auc: 0.7763
Epoch 2/100
588001/588001 [==============================] - 24s 40us/sample - loss: 0.4161 - auc: 0.7571 - val_loss: 0.4040 - val_auc: 0.7825
Epoch 3/100
588001/588001 [=================

In [18]:
print("Overall AUC={}".format(metrics.roc_auc_score(train.target.values, oof_preds)))

Overall AUC=0.7835388190012328


In [19]:
test_preds /= 50
test_ids = test.id.values
print("Saving submission file")
submission = pd.DataFrame.from_dict({
    'id': test_ids,
    'target': test_preds
})
submission.to_csv("data/submission-22.csv", index=False)

Saving submission file
